This first part of preprocessing consists of the so-called electrode correlation correction. The mean of the signals coming from the same electrode was calculated and subtracted from individual signal of each channel belonging to that electrode.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
import csv
from scipy.optimize import curve_fit
from scipy.fft import fft
from scipy import signal
from scipy.signal import iirnotch, lfilter
from scipy.signal import butter, filtfilt
import os
import networkx as nx
from scipy import stats
from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.signal import find_peaks
import statistics
from scipy.signal import peak_widths
from sklearn.metrics import mutual_info_score
import matplotlib.patches as mpatches
from scipy.signal import butter, lfilter
from collections import defaultdict

In [ ]:
# Percorso della cartella contenente i file CSV
input_dir = 'lesscolumns'
output_dir = 'preprodata'

# Verifica se la cartella di output esiste, altrimenti la crea
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Dizionario per memorizzare i valori per ogni gruppo
grouped_values = defaultdict(list)
file_data = {}

# Legge tutti i file nella directory e li raggruppa
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        filepath = os.path.join(input_dir, filename)
        df = pd.read_csv(filepath, header=None, low_memory=False)
        nome_colonna = df.iloc[0, 0]
        key = nome_colonna[0]
        grouped_values[key].extend(df.iloc[1:750001, 0].astype(float).tolist())  # Read the first 750,000 rows
        file_data[filename] = df[0:750001]

# Calcola la media per ogni gruppo
group_means = {}
for key, values in grouped_values.items():
    group_means[key] = sum(values) / len(values)
    print(f"Media per il gruppo '{key}': {group_means[key]}")

# Applica la correzione e salva i nuovi file
for filename, df in file_data.items():
    nome_colonna = df.iloc[0, 0]
    key = nome_colonna[0]  # Prende la prima lettera della prima riga
    mean_value = group_means[key]

    # Sottrae la media dai valori del file
    df.iloc[1:, 0] = df.iloc[1:, 0].astype(float) - mean_value

    # Salva il nuovo file nella cartella di output
    new_filepath = os.path.join(output_dir, filename)
    df.to_csv(new_filepath, index=False, header=False)

print("Elaborazione completata.")

<ipython-input-3-b055866d77b5>:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, header=None)
<ipython-input-3-b055866d77b5>:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, header=None)
<ipython-input-3-b055866d77b5>:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, header=None)
<ipython-input-3-b055866d77b5>:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, header=None)
<ipython-input-3-b055866d77b5>:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, header=None)
<ipython-input-3-b055866d77b5>:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or se

ValueError: could not convert string to float: '-2.4412676435263563e-'

In [ ]:
#CONTROLLO CHE LA DIFFERENZA TRA I FILE CORRETTI E QUELLI ORIGINALI SIA EFFETTIVAMENTE LA MEDIA CHE VOLEVO TOGLIERE
# Percorso delle due cartelle contenenti i file CSV
folder1 = 'lesscolumns'
folder2 = 'preprodata'

# Leggi i nomi dei file in entrambe le cartelle
files1 = set(f for f in os.listdir(folder1) if f.endswith('.csv'))
files2 = set(f for f in os.listdir(folder2) if f.endswith('.csv'))

# Trova i file comuni alle due cartelle
common_files = files1.intersection(files2)

# Calcola e stampa la differenza tra i valori dei file comuni
for filename in common_files:
    filepath1 = os.path.join(folder1, filename)
    filepath2 = os.path.join(folder2, filename)

    df1 = pd.read_csv(filepath1, header=None)
    df2 = pd.read_csv(filepath2, header=None)

    # Assicurati che entrambi i DataFrame abbiano lo stesso numero di righe
    if df1.shape[0] != df2.shape[0]:
        print(f"Il file {filename} ha un numero diverso di righe nelle due cartelle.")
        continue

    # Calcola la differenza tra i valori della prima colonna a partire dalla seconda riga
    diff = df1.iloc[1:10, 0].astype(float) - df2.iloc[1:10, 0].astype(float)

    print(f"Differenze per il file {filename}:")
    print(diff.tolist())
    print()

print("Elaborazione completata.")



Differenze per il file 2_J3.csv:
[1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303438e-05]

Differenze per il file 1_J2.csv:
[1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303432e-05, 1.4277289784303438e-05, 1.4277289784303438e-05, 1.4277289784303438e-05, 1.4277289784303438e-05, 1.4277289784303438e-05, 1.4277289784303438e-05]

Differenze per il file 28_L15.csv:
[1.4611987218481496e-05, 1.4611987218481496e-05, 1.4611987218481496e-05, 1.4611987218481496e-05, 1.4611987218481496e-05, 1.4611987218481496e-05, 1.4611987218481496e-05, 1.4611987218481496e-05, 1.4611987218481496e-05]

Differenze per il file 21_L8.csv:
[1.4611987218481496e-05, 1.4611987218481489e-05, 1.4611987218481489e-05, 1.4611987218481489e-05, 1.4611987218481489e-05, 1.4611987218481489e-05, 1.4611987218481489e-05, 1.4611987218481493e-05, 1.4611987218481489